### import

In [7]:
import pandas as pd
import numpy as np
from os.path import join
import functions
import warnings
import re
warnings.simplefilter("ignore")

### 일반

In [16]:
# ################################
company = "대성"
최종조회일 = "250930"
데이터종류 = "메모"
# 파일 하나만 추가할 경우 파일명만(확장자x), 여러개일 경우 ""
fn_excel = "메모조회새창_971213~250930"
# ################################
if 데이터종류 == "입금" : 
    최초조회일 = "160119" if company == "솔림" else "150824"
    deposit_dtype = functions.deposit_dtype
    기간칼럼명 = "입금일"
elif 데이터종류 == "메모" : 
    최초조회일 = "930426" if company == "솔림" else "971213"
    deposit_dtype = functions.memo_dtype
    기간칼럼명 = "등록일자"
# ################################
path_excel = join(r"D:\3.자산\전산 dataset",company,"기간축적데이터", 데이터종류)
path_parquet = join(r"D:\3.자산\전산 dataset",company, "기간축적데이터", 데이터종류, "parquet")
fn_write = company + 데이터종류+"전체_" + 최초조회일 + "~" + 최종조회일 +".parquet"

In [22]:
# parquet 파일 읽기
path_parquet_이전기간전체파일 = functions.키워드로파일명찾기(path_parquet, 포함키워드=company+데이터종류+"전체", 여러파일허용=True, 전체경로=True)
if isinstance(path_parquet_이전기간전체파일, list) : # 파일 여러개
    df_ori = pd.read_parquet(path_parquet_이전기간전체파일[-1])
elif isinstance(path_parquet_이전기간전체파일, str) : # 파일 한 개
    df_ori = pd.read_parquet(path_parquet_이전기간전체파일)
else : # 파일 없음
    df_ori = pd.DataFrame(None)
if not df_ori.empty :
    print(f'df_ori(이전기간 parquet파일) 기간 : {df_ori[f"{기간칼럼명}"].min()}~{df_ori[f"{기간칼럼명}"].max()}')

# 엑셀 파일 읽기
if fn_excel =="":
    fnlist = functions.키워드로파일명찾기(path_excel, 포함키워드=데이터종류+"조회새창", 여러파일허용=True)
    df = pd.DataFrame(None)
    for fn in fnlist :
        print(f'{fn} 읽기 시작')
        df = pd.concat([df, pd.read_excel(fn, dtype=deposit_dtype)])
else : 
    fnlist = functions.키워드로파일명찾기(path_excel, fn_excel+".xlsx")
    df = pd.read_excel(fnlist, dtype=deposit_dtype)


# 공지메모의 등록일자 바로잡기 : 엑셀만 하면 됨
if 데이터종류 == "메모" : 
    mask = df["등록일자"].str.contains("★", na=False)
    df.loc[mask, "등록일자"] = (
        df.loc[mask, "등록일자"]
        .str.extract(r"(\d{4}-\d{2}-\d{2})")[0]
    )

print(f'df(엑셀파일) 기간 : {df[f"{기간칼럼명}"].min()}~{df[f"{기간칼럼명}"].max()}')

# 합치기
df_concat = pd.concat([df_ori, df])

print("df_concat(병합파일) 기간확인", df_concat[f"{기간칼럼명}"].min(), df_concat[f"{기간칼럼명}"].max())

포함키워드:대성메모전체, 제외키워드 조건을 만족하는 파일이 없습니다.
df(엑셀파일) 기간 : 1997-12-13 00:00:00~2025-09-30 16:38:30
df_concat(병합파일) 기간확인 1997-12-13 00:00:00 2025-09-30 16:38:30


In [24]:
# parquet 저장 : 통합본 읽을 때 정렬 및 인덱스 전처리 안 함에 유의
functions.to_parquet(df_concat, join(path_parquet, fn_write))

### 나의사건

#### 나의 사건 전체 목록 만들기/읽기

In [17]:
# ################################
company = "솔림"
지점 = "광주"
최종조회일 = "250831" # 월말 자산정리일 기준으로 작성, 아닌 경우 df_law 읽을 때 따로 날짜 지정해야
데이터종류 = "나의사건"
나의사건_dtype = str
# 파일 하나만 추가할 경우 파일명만(확장자x), 여러개일 경우 ""
fn_excel = None  # "" : 여러파일, "파일명" : 단일파일, None : 기존 parquet만 읽기
최초조회일 = "170802" if company == "솔림" else ""
# ################################
path_excel = join(r"D:\3.자산\전산 dataset",company,"기간축적데이터", 데이터종류)
path_parquet = join(r"D:\3.자산\전산 dataset",company, "기간축적데이터", 데이터종류, "parquet")
fn_write = company + 데이터종류+"전체_" + 최초조회일 + "~" + 최종조회일 +".parquet"


# parquet 파일 읽기
path_parquet_files = functions.키워드로파일명찾기(path_parquet, 포함키워드=company+데이터종류+"전체", 여러파일허용=True, 전체경로=True)
if isinstance(path_parquet_files, str) :
    path_parquet_fn = path_parquet_files
else :
    path_parquet_fn = path_parquet_files[-1]

try : 
    df_ori = pd.read_parquet(path_parquet_fn)
except : # 파일이 없는 경우
    df_ori = pd.DataFrame(None)

# 엑셀 파일 읽기
if fn_excel =="":
    df = pd.DataFrame(None)
    for 사건진행구분 in ["확정사건", "완료사건","진행사건"] : 
        
        fnlist = functions.키워드로파일명찾기(path_excel, 포함키워드=지점+" "+사건진행구분, 여러파일허용=True)
        
        if isinstance(fnlist, str) : # 파일이 한 건
            print(f'{fnlist} 읽기 시작')
            df_temp = pd.read_excel(fnlist, dtype=나의사건_dtype)
            df_temp["사건진행구분"] = 사건진행구분
            df = pd.concat([df, df_temp])
        else :
            for fn in fnlist :
                print(f'{fn} 읽기 시작')
                df_temp = pd.read_excel(fn, dtype=나의사건_dtype)
                df_temp["사건진행구분"] = 사건진행구분
                df = pd.concat([df, df_temp])
elif fn_excel == None :
    df = df_ori.copy()
else : 
    fnlist = functions.키워드로파일명찾기(path_excel, fn_excel+".xlsx")
    사건진행구분 = re.search("(진행|확정|완료)사건", "광주 확정사건 220701~250831")[0]
    df = pd.read_excel(fnlist, dtype=나의사건_dtype)
    df["사건진행구분"] = 사건진행구분

if fn_excel!=None :
    df["지점"] = 지점

#### 채무자키 매칭

In [28]:
# 채무자키 넣기
    # 법조치 읽기
fn_law = functions.키워드로파일명찾기(join(r'D:\3.자산\전산 dataset\솔림',최종조회일,"parquet"),"법조치조회새창", 여러파일허용=False)
df_law = pd.read_parquet(fn_law)
    # 개인회생 전체파일 읽기
fn_rehabilitation = functions.키워드로파일명찾기(join(r'D:\3.자산\전산 dataset\솔림',최종조회일,"parquet"),r"\[상태DC\]개인회생", 여러파일허용=False)
df_rehabilitation = pd.read_parquet(fn_rehabilitation)

In [34]:
df_law.columns[df_law.columns.str.contains('주민')]

Index(['주민번호'], dtype='object')

In [ ]:
# 1. df_law 후보 만들기
change_cols_name_dict = {"관할법원":"법원", "관련사건법원":"법원", "관련법조치관할법원": "법원", "관련사건번호":"사건번호", "관련법조치사건번호":"사건번호", "주민번호":"차주주민번호", "주민번호인":"차주주민번호"}

cand1 = df_law[["관할법원", "사건번호", "채무자키","주민번호"]].rename(columns=change_cols_name_dict)
cand2 = df_law[["관련사건법원", "관련사건번호", "채무자키","주민번호"]].rename(columns=change_cols_name_dict)
cand3 = df_law[["관련법조치관할법원", "관련법조치사건번호", "채무자키","주민번호"]].rename(columns=change_cols_name_dict)

cand4 = df_rehabilitation[["관할법원","사건번호","채무자키","주민번호인"]].rename(columns=change_cols_name_dict)

# 2. 전부 합치기
df_law_candidates = pd.concat([cand1, cand2, cand3, cand4], ignore_index=True)


# 후처리 
    # 정렬
df_law_candidates = df_law_candidates.sort_values(['법원','사건번호','채무자키'])
    
    # 법원-사건번호-채무자키 중복건 삭제
df_law_candidates = df_law_candidates.drop_duplicates(subset=['법원','사건번호','채무자키'], keep='first')

    # 관할법원, 사건번호 없는 건은 삭제
df_law_candidates = df_law_candidates.replace("", np.nan)
print('법원사건번호에 빈값 있는 행:',len(df_law_candidates))
df_law_candidates = df_law_candidates.dropna(subset=["법원", "사건번호"], how='any')
print('빈값 삭제 후',len(df_law_candidates))

# [dc] 동일 사건에, 차주 주민번호 다른 건 확인
dup_cases = (
    df_law_candidates
    .groupby(["법원", "사건번호"])["차주주민번호"]
    .nunique()
    .reset_index()
    .query("차주주민번호 > 1")   # 고유 주민번호 개수가 2개 이상인 경우만
)

    # 중복등록이 맞음
if company == "솔림" : 
    data = [
        ["광주지방법원","2003가단16998"],
        
    ]
    중복확인된사건 = pd.DataFrame(data, columns=["법원","사건번호"])



# 해당 케이스의 원본 데이터 뽑기
problem_rows = df_law_candidates.merge(dup_cases[["법원","사건번호"]], on=["법원","사건번호"], how="inner")
functions.save_excel_with_explain(problem_rows, "차주중복등록사건.xlsx", key_columns_no=2)

법원사건번호에 빈값 있는 행: 172322
빈값 삭제 후 130451
★ 차주중복등록사건 252건 엑셀출력



In [60]:
pd.DataFrame([["광주지방법원","2003가단16998"]], columns=["법원","사건번호"])

,법원,사건번호
0,광주지방법원,2003가단16998


In [ ]:
# # 3. 중복제거: 법원+사건번호 기준, 채무자키는 첫 번째만
# df_law_candidates = df_law_candidates.drop_duplicates(subset=["법원","사건번호"])

# # 4. df와 merge
# df = df.merge(df_law_candidates, on=["법원","사건번호"], how="left")

# # 5. 못 찾은 건 빈 문자열
# df["채무자키"] = df["채무자키"].fillna("")


,법원,사건번호,채무자키,차주주민번호
0,수원지방법원,2025타채134613,20428861,401108-1052413
1,수원지방법원,2025타채134613,20500063,821210-1122013
2,광주지방법원,2024하면5081,20428934,640311-1495617
3,광주지방법원,2024하면5081,20429840,571115-2495634
4,전주지방법원,2024하면296,20413618,640522-2661011
...,...,...,...,...
426,울산지방법원,2019개회11603,20514978,730209-2785414
427,울산지방법원,2019개회11597,20515427,710428-1453110
428,울산지방법원,2019개회11597,20514978,730209-2785414
429,수원회생법원,2019개회16391,20408300,681020-2668033


In [ ]:
# # 엑셀에 저장
# with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
#     df.to_excel(writer, sheet_name = "집행문", index=False)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [50]:
# 합치기
df_concat = pd.concat([df_ori, df])

# 불요칼럼 삭제, 정렬->중복검사
    # 불요칼럼
불요칼럼 = ["번호"]
df_concat = df_concat.drop(columns=불요칼럼)

df_concat["법원"] = df_concat.법원.str.replace("지법", "지방법원")

    # 정렬 : 최근 사건이 먼저, 확정사건이 먼저 (확정사건을 추가한 경우 진행사건이 삭제될 수 있도록)
df_concat["접수일자"] = pd.to_datetime(df_concat["접수일자"], errors='coerce')
df_concat["확정일자"] = pd.to_datetime(df_concat["확정일자"], errors='coerce')
df_concat = df_concat.sort_values(by=["확정일자", "접수일자", "법원","사건번호"], ascending=[False, False, True,True])

    # 중복확인 및 삭제
functions.display_with_explain(df_concat[df_concat.duplicated(subset=["법원", "사건번호"], keep=False)], "중복사건")
df_concat = df_concat.drop_duplicates(subset=["법원", "사건번호"])

    # 새로운 인덱싱
df_concat.reset_index(inplace=True, drop=True)

    # 칼럼순서
cols = ['지점', '법원', '사건번호', '재판부', '사건지위', '원고', '피고', '사건진행구분','확정일자', '접수일자',
       '기일시간', '기일장소']
df_concat = df_concat[cols]

# parquet 저장 : 통합본 읽을 때 정렬 및 인덱스 전처리 안 함에 유의
functions.to_parquet(df_concat, join(path_parquet, fn_write))

☆ 중복사건 없음

